In [5]:
%pip install pandas
%pip install -r requirements.txt

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: c:\Users\ADMIN\Programming\ML\Hacks\vk-hack-cats-and-rabbits\venv\Scripts\python.exe -m pip install --upgrade pip


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached contourpy-1.3.2-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp312-cp312-win_amd64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cache


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: c:\Users\ADMIN\Programming\ML\Hacks\vk-hack-cats-and-rabbits\venv\Scripts\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [2]:
# Загрузка данных (предположим, что данные в CSV)
df = pd.read_csv("./datasets/train/train.csv") # для train

In [3]:

# Удаляем индекс, так как он не несёт информации для модели
X = df.drop(['index', 'target'], axis=1)  # Все фичи (feature_1 ... feature_1366)
y = df['target']  # Целевая переменная

X, y

(        feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
 0        0.128367        1.0        1.0        1.0        1.0        4.0   
 1        0.258685        1.0        1.0        1.0        0.0        7.0   
 2        0.407738        1.0        1.0        1.0        2.0        7.0   
 3        0.439134        1.0        1.0        1.0        1.0        2.0   
 4        0.590481        1.0        1.0        1.0        0.0        1.0   
 ...           ...        ...        ...        ...        ...        ...   
 177853   0.660324        1.0        1.0        1.0        0.0        1.0   
 177854   0.222214        0.0        1.0        1.0        1.0        4.0   
 177855   0.545399        0.0        1.0        1.0        0.0        2.0   
 177856   0.393150        1.0        1.0        1.0        0.0        1.0   
 177857   0.395272        1.0        1.0        1.0        1.0        2.0   
 
         feature_6  feature_7  feature_8  feature_9  ...  feature_1357  \


In [4]:
indexes = df['index']

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=200)),
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

X_selected = pipe.fit_transform(X, y)
selector = pipe.named_steps['kbest']

print(f"Исходное число признаков: {X.shape[1]}")
print(f"Число признаков после отбора: {X_selected.shape[1]}")

Исходное число признаков: 1367
Число признаков после отбора: 200


In [4]:
# 2. Масштабирование данных (важно для линейной регрессии)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42
)


In [ ]:
# 4. Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# 5. Предсказание и оценка качества
y_pred = model.predict(X_test)
# y_pred_proba_lin_reg = model.decision_function(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
# нормализуем значения для roc_auc
y_proba = 1 / (1 + np.exp(-y_pred)) # преобразование через сигмойду
roc_auc = roc_auc_score(y_test, y_proba)

print(f"\nMSE: {mse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# (Опционально) Вывод коэффициентов модели
if X_selected.shape[1] <= 20:  # Не выводить для слишком большого числа фич
    print("\nКоэффициенты модели:")
    # for feature, coef in zip(X.columns[selector.get_support()], model.coef_):
    #     print(f"{feature}: {coef:.4f}")
else:
    print(f"\nМодель имеет {X_selected.shape[1]} коэффициентов (слишком много для вывода).")


MSE: 0.0171
R2 Score: 0.0016
ROC-AUC Score: 0.6536

Модель имеет 400 коэффициентов (слишком много для вывода).


In [42]:
y_pred

array([ 0.01629677,  0.00887584,  0.00911322, ...,  0.0150569 ,
       -0.0037384 , -0.00022041], shape=(53358,))

Так.... Из полученных метрик следует что вроде бы по MSE результат довольно терпимый и среднеквадратичная ошибка не слишком большая.
Касаемо метрики ROC-AUC Score получаем 0.6536 что является лучше чем убычное угадывание(подбрасывание монетки).
Из инересного мы не должны использовать LinearRegression, так как она может выдавать значения за пределами [0, 1], но после регуляризации значений, они могут быть использованиы для вычисления метрики. 

Попробуем RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)


In [13]:
y_pred_proba

array([0.  , 0.04, 0.03, ..., 0.  , 0.03, 0.  ], shape=(53358,))

In [ ]:
# generate output data
X_true_test = pd.read_csv('./datasets/test/test.csv')
X_true_test

,index,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_1357,feature_1358,feature_1359,feature_1360,feature_1361,feature_1362,feature_1363,feature_1364,feature_1365,feature_1366
0,124365,0.719921,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,177140,0.123122,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,106632,0.384612,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,17630,0.452527,0.0,1.0,1.0,1.0,4.0,5.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,97189,0.379611,1.0,1.0,1.0,2.0,11.0,3.0,4.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76220,55074,0.325234,0.0,1.0,1.0,1.0,3.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76221,12771,0.227776,1.0,1.0,1.0,1.0,3.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76222,80999,0.459919,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76223,31517,0.667716,1.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [47]:
print(f"ROC_AUC_SCORE is {roc_auc:.4f}")

ROC_AUC_SCORE is 0.6261


Прогресс, модель достигла случайного угадывания
(последние данные ROC_AUC_SCORE is 0.6261, это даже лучше чем просто уадывание по среднему значению)

In [32]:
mse = mean_squared_error(y_test, y_pred_proba)
print(f'MSE: {mse:.2f}')

MSE: 0.02


Небольшой апдейт для структуры модели

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)


In [33]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6221


А что если заменить RandomForsetClassifier на LogisticRegression

In [36]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [37]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6473


Вау, метрика подросла)


Теперь попробуем GradientBoostingClassifier


In [40]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', GradientBoostingClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [41]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6796


Попробуем еще одну модель SVC

In [6]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True, kernel='rbf', C=1.0, decision_function_shape='ovr'))
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [7]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.5873
